In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_syste

In [ ]:
!pip install dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.37
    Uninstalling SQLAlchemy-2.0.37:
      Successfully uninstalled SQLAlchemy-2.0.37
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# Load datasets (adjust paths as needed)
train_df = pd.read_csv("/content/AWT_train.csv")
dev_df = pd.read_csv("/content/AWT_dev.csv")
#test_df = pd.read_csv("/content/AWT_test_without_labels.csv")


In [ ]:
train_df.describe()

,Text,Class
count,2790,2790
unique,2779,3
top,"அழகு ராஜா, மணிகண்டன், செல்வா, கார்த்திக், ரமேஷ...",Non-Abusive
freq,3,1424


In [ ]:
dev_df.describe()

,Text,Class
count,598,598
unique,598,2
top,"மேடம் இந்த கேசெல்லாம் விசாரிச்சா, சொல்வதெல்லாம...",Non-Abusive
freq,1,320


In [ ]:
test_df.describe()

,id
count,598.000000
mean,299.500000
std,172.772008
min,1.000000
25%,150.250000
50%,299.500000
75%,448.750000
max,598.000000


In [ ]:
# Standardize labels by merging 'abusive' into 'Abusive'
train_df['Class'] = train_df['Class'].replace({'abusive': 'Abusive'})
dev_df['Class'] = dev_df['Class'].replace({'abusive': 'Abusive'})
# Calculate the distribution of labels
label_distribution = train_df['Class'].value_counts()
label_distribution1 = dev_df['Class'].value_counts()
label_distribution2 = test_df['Text'].value_counts()

# Calculate total samples
total_samples = train_df.shape[0]
total_samples1 = dev_df.shape[0]
total_samples2 = test_df.shape[0]
# Create a DataFrame to display the summary
data_summary = pd.DataFrame({
    "Total Samples": [total_samples],
    "Abusive": [label_distribution.get('Abusive', 0)],
    "Non-Abusive": [label_distribution.get('Non-Abusive', 0)]
})
data_summary1 = pd.DataFrame({
    "Total Samples": [total_samples1],
    "Abusive": [label_distribution1.get('Abusive', 0)],
    "Non-Abusive": [label_distribution1.get('Non-Abusive', 0)]
})
data_summary2 = pd.DataFrame({
    "Total Samples": [total_samples2],
    # "Abusive": [label_distribution.get('Abusive', 0)],
    # "Non-Abusive": [label_distribution.get('Non-Abusive', 0)]
})
print("Dataset Summary:")
print(data_summary)
print(data_summary1)
print(data_summary2)

Dataset Summary:
   Total Samples  Abusive  Non-Abusive
0           2790     1366         1424
   Total Samples  Abusive  Non-Abusive
0            598      278          320
   Total Samples
0            598


In [ ]:
# Calculate the actual split from your data
train_split = train_df['Class'].value_counts()
dev_split = dev_df['Class'].value_counts()

# Display the data distribution
data_split_actual = {
    "Dataset": ["Train", "Dev"],
    "Abusive": [train_split.get("abusive", 0), dev_split.get("abusive", 0)],
    "Non-abusive": [train_split.get("non-abusive", 0), dev_split.get("non-abusive", 0)],
}

# Create a DataFrame to display
split_actual_df = pd.DataFrame(data_split_actual)
print(split_actual_df)


  Dataset  Abusive  Non-abusive
0   Train        1            0
1     Dev        0            0


In [ ]:
dev_df.isnull().sum()

,0
Text,0
Class,0


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(dev_df)
#test_dataset = Dataset.from_pandas(test_df)

In [ ]:
from sklearn.model_selection import train_test_split

# Encode the labels
train_df['Class'] = train_df['Class'].map({'Abusive': 1, 'abusive': 1, 'Non-Abusive': 0})

In [ ]:
# Rename the 'Class' column to 'Abusive'
dev_df.rename(columns={'Class': 'Abusive'}, inplace=True)

# Standardize the labels by merging 'abusive' into 'Abusive'
dev_df['Abusive'] = dev_df['Abusive'].replace({'abusive': 'Abusive'})

# Reanalyze the distribution of labels
corrected_label_distribution1 = dev_df['Abusive'].value_counts()

print("Corrected Label Distribution:")
print(corrected_label_distribution1)


Corrected Label Distribution:
Abusive
Non-Abusive    320
Abusive        278
Name: count, dtype: int64


In [ ]:
# Rename the 'Class' column to 'Abusive'
train_df.rename(columns={'Class': 'Abusive'}, inplace=True)

# Standardize the labels by merging 'abusive' into 'Abusive'
train_df['Abusive'] = train_df['Abusive'].replace({'abusive': 'Abusive'})

# Reanalyze the distribution of labels
corrected_label_distribution = train_df['Abusive'].value_counts()

print("Corrected Label Distribution:")
print(corrected_label_distribution)

Corrected Label Distribution:
Series([], Name: count, dtype: int64)


In [ ]:
train_dataset.features


{'Text': Value(dtype='string', id=None),
 'Class': Value(dtype='string', id=None)}

In [ ]:
def encode_labels(example):
    label_map = {'Abusive': 1, 'Non-Abusive': 0}  # Match the actual label casing
    example['Class'] = label_map[example['Class']]  # Map directly using the correct case
    return example

def normalize_labels(example):
    # Standardize 'abusive' to 'Abusive'
    if example['Class'].strip().lower() == 'abusive':
        example['Class'] = 'Abusive'
    return example

# Apply the normalization
train_dataset = train_dataset.map(normalize_labels)
# Apply the mapping
train_dataset = train_dataset.map(encode_labels)


Map:   0%|          | 0/2790 [00:00<?, ? examples/s]

Map:   0%|          | 0/2790 [00:00<?, ? examples/s]

In [ ]:
train_dataset['Class']

[0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,


In [ ]:
from sklearn.model_selection import train_test_split
# Split into training and validation sets (80%-20% split)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_dataset['Text'], train_dataset['Class'], test_size=0.2, random_state=42
)



`dccdccvvv`

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments



# Tokenizer and model
model_name = "ai4bharat/indic-bert"  # Or "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["Text"], truncation=True, padding=True, max_length=128)



train_dataset = train_dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/2790 [00:00<?, ? examples/s]

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch

# Step 1: Load Pre-trained Tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

# Step 2: Tokenize Text Data
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

# Convert data to Hugging Face Dataset format
train_data = Dataset.from_dict({
    "Text": train_texts,
    "labels": [int(label) for label in train_labels]
})
val_data = Dataset.from_dict({
    "Text": val_texts,
    "labels": [int(label) for label in val_labels]
})


# Apply tokenization
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Set format for PyTorch
def format_data(data):
    return data.with_format("torch", columns=["input_ids", "attention_mask", "labels"])

train_data = format_data(train_data)
val_data = format_data(val_data)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/2232 [00:00<?, ? examples/s]

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

In [ ]:
# Check data format
print("Sample Input IDs:", train_data[0]["input_ids"])
print("Sample Label:", train_data[0]["labels"])


Sample Input IDs: tensor([     0,  29883,  49381, 203549,  52709,  47128,   3686,  22162,  94116,
          6935,  52709, 104542,   4864, 101514,      5,  19376,  51153,  30240,
             6, 144038, 144940,   3791,   4864,  12949,      2,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1

In [ ]:
print(train_dataset[0])  # View the first example after tokenization

{'Text': 'இதல்லம் ஒரு தீர்ப்பு நாட்டாமை தீர்ப்பை மற்று', 'Class': 0}


In [ ]:
# Step 3: Load Pre-trained Model
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2)
from transformers import EarlyStoppingCallback

early_stopping = EarlyStoppingCallback(early_stopping_patience=3)


# Step 4: Define Trainer and Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",  # Evaluate during training
    learning_rate=2e-5,          # Tune learning rate
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,          # Increase/decrease epochs
    weight_decay=0.01,           # Regularization
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=100,
    fp16=True
)
from transformers import Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    callbacks=[early_stopping]# Handles tokenization
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=lambda p: {
        "accuracy": (p.predictions.argmax(-1) == p.label_ids).mean()
    }
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-10-65cf5ed0b328>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    predictions, labels = pred
    predictions = predictions.argmax(axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
import numpy as np

def predict_with_threshold(trainer, dataset, threshold=0.6):
    predictions = trainer.predict(dataset)
    logits = predictions.predictions
    probabilities = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    return (probabilities[:, 1] > threshold).astype(int)


In [ ]:
# Step 6: Evaluate the Model
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.5773317217826843, 'eval_accuracy': 0.7903225806451613, 'eval_runtime': 4.4433, 'eval_samples_per_second': 125.582, 'eval_steps_per_second': 7.877, 'epoch': 5.0}


In [ ]:
# Testing with another dataset
test_data = pd.read_csv('/content/AWT_dev.csv')

test_data.isnull().sum()


NameError: name 'pd' is not defined

In [ ]:
test_data['Class'] = test_data['Class'].map({'Abusive': 1, 'Non-Abusive': 0})
test_texts = test_data['Text'].astype(str).tolist()
test_labels = test_data['Class'].tolist()

In [ ]:
test_labels = torch.tensor(test_labels, dtype=torch.long)  # Convert labels to int64
test_dataset = TextDataset(test_encodings, test_labels)


<ipython-input-39-54d1647fe5fc>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels, dtype=torch.long)  # Convert labels to int64
<ipython-input-38-4d281384b959>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)  # Ensure labels are int64


In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)  # Ensure labels are int64

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]  # Already a tensor with dtype=torch.long
        return item


# Create the dataset
test_dataset = TextDataset(test_encodings, test_labels)


<ipython-input-40-4d281384b959>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)  # Ensure labels are int64


In [ ]:
import torch

# Convert class labels to numerical values
test_data['Class'] = test_data['Class'].map({'Abusive': 1, 'Non-Abusive': 0})

# Extract text and labels
test_texts = test_data['Text'].astype(str).tolist()
test_labels = test_data['Class'].tolist()

# Tokenize the test data correctly
test_encodings = tokenizer(test_texts, padding="max_length", truncation=True, return_tensors="pt")

# Convert labels to tensor
test_labels = torch.tensor(test_labels)

# Create the dataset (assuming TextDataset takes encodings and labels)
test_dataset = TextDataset(test_encodings, test_labels)


<ipython-input-40-4d281384b959>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)  # Ensure labels are int64


In [ ]:
# Predictions on the test set

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=1)

<ipython-input-40-4d281384b959>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:

# Classification report
print(classification_report(test_labels, preds, target_names=['Non-Abusive', 'Abusive']))


In [ ]:
# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(test_labels, preds))

In [ ]:
# Load the unlabeled dataset
unlabeled_data = pd.read_csv('/content/AWT_test_without_labels.csv')

# Extract the 'Text' column
unlabeled_texts = unlabeled_data['Text'].astype(str).tolist()

# Tokenize the unlabeled dataset
unlabeled_encodings = tokenize_function(unlabeled_texts)

In [ ]:
# Create a dataset object for the unlabeled data
class UnlabeledDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

unlabeled_dataset = UnlabeledDataset(unlabeled_encodings)

In [ ]:
# Predict on the unlabeled data
unlabeled_predictions = trainer.predict(unlabeled_dataset)
unlabeled_preds = np.argmax(unlabeled_predictions.predictions, axis=1)


In [ ]:
# Map predictions back to labels (0: 'Non-Abusive', 1: 'Abusive')
#unlabeled_data['Labels'] = unlabeled_preds
unlabeled_data['Labels'] = unlabeled_data['Predicted_Class'].map({0: 'Non-Abusive', 1: 'Abusive'})


In [ ]:
unlabeled_data.head(5)

In [ ]:
unlabeled_data.drop('Text',
  axis='columns', inplace=True)

In [ ]:
unlabeled_data.head(5)

In [ ]:
# Save the results to a CSV file
unlabeled_data.to_csv('/content/results/MSM_CUET_Tamil_run1.csv', index=False)

print("Predictions saved to /content/results/VEL_Tamil_run1.csv")

In [ ]:
# Get confidence scores
confidence_scores = torch.softmax(torch.tensor(unlabeled_predictions.predictions), dim=1).numpy()
max_confidences = np.max(confidence_scores, axis=1)

# Add confidence scores to the DataFrame
unlabeled_data['Confidence'] = max_confidences

# Save results with confidence scores
unlabeled_data.to_csv('/content/results/predicted_with_confidence.csv', index=False)


er ag porjonto but kno run hocche na ke jane
